In [1]:
import Pkg
Pkg.activate(raw"D:/DTU/OR/VS_codes/FCB")
Pkg.status()

  Activating project at `D:\DTU\OR\VS_codes\FCB`


Status `D:\DTU\OR\VS_codes\FCB\Project.toml`
  [336ed68f] CSV v0.10.15
  [5ae59095] Colors v0.13.1
  [a9b2a840] Complementarity v0.9.0
  [a93c6f00] DataFrames v1.8.1
⌃ [2e9cd046] Gurobi v1.7.6
  [87dc4568] HiGHS v1.20.1
⌃ [7073ff75] IJulia v1.32.1
  [033835bb] JLD2 v0.6.3
⌃ [4076af6c] JuMP v1.29.2
  [b964fa9f] LaTeXStrings v1.4.0
  [15e1cf62] NPZ v0.4.3
  [f5f7c340] PATHSolver v1.7.9
⌃ [b1d3bc72] Pathfinder v0.9.26
⌃ [91a5bcdd] Plots v1.41.1
⌃ [08abe8d2] PrettyTables v3.1.0
  [438e738f] PyCall v1.96.4
  [10745b16] Statistics v1.11.1
⌃ [2913bbd2] StatsBase v0.34.8
  [f3b207a7] StatsPlots v0.15.8
  [3a884ed6] UnPack v1.0.2
  [fdbf4ff8] XLSX v0.10.4
  [37e2e46d] LinearAlgebra v1.12.0
Info Packages marked with ⌃ have new versions available and may be upgradable.


In [2]:
using JuMP, Plots, HiGHS,  Statistics, LinearAlgebra, LaTeXStrings, DataFrames, CSV, XLSX, StatsPlots, NPZ, UnPack, JLD2, Random

include("constants.jl")

0:100:10000

In [4]:
include("opt_models.jl")
include("outputs_processing.jl")

preprocess_scenario_files (generic function with 1 method)

In [5]:
results_RI_NI = reserve_RI_NI(NI_up_og, NI_down_og, Cf_Rup_og, Cf_Rdown_og)
results_RI = reserve_RI(Cf_Rup_og, Cf_Rdown_og)
rf_up, rf_down = results_RI_NI[end].rf_up, results_RI_NI[end].rf_down
q_up, q_down = get_your_qs(results_RI_NI[end], results_RI[end])
Lambda_r_up, Lambda_r_down, fig_down = merit_order_price_reserve(rf_up, rf_down, Cf_Rup_og, Cf_Rdown_og; dir = "down regulation")
_,_, fig_up = merit_order_price_reserve(rf_up, rf_down, Cf_Rup_og, Cf_Rdown_og; dir = "up regulation")
Lambda_R = (Lambda_r_up, Lambda_r_down)
R_bids = [rf_up, rf_down]

2-element Vector{Vector{Float64}}:
 [30.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0]
 [0.0, 0.0, 0.0, 0.0, 40.0, 40.0, 40.0, 40.0]

In [6]:
file_list = ["seasonal_scenarios_0125.csv"]

Pw, centers, Prob, nS_all = preprocess_scenario_files(file_list; season="Spring")         # Considering that they all have the same uncertainty

hour = 1
prob = Vector{Float64}(undef, nS_all)
prob = round.(Prob[1][:, hour], digits = 5)     # in this case we prob is only constituted of 1 element

k = 10
fold_size = div(nS_all, k)
Pw_hour = Pw[hour]
n_hours = size(Pw)[1]

24

In [7]:
# Benchmark 
#fold 1 by setting k = 1
fold = 1
res_BM =  run_kfold_simulation("Benchmark", Pw, nS_all, prob, 0, 0, fold, fold_size, 
                            rf_up, rf_down,NI_up_og, NI_down_og, Cf_Rup_og,  Cf_Rdown_og, R_bids, Lambda_R, hour)

(profits = (pol_in = [[2325.96, 169.37165337851323, -0.0, -0.0]; [0.0, 2495.3316533785137, -0.0, -0.0]; … ; [3590.77, 261.3025822204504, -0.0, -0.0]; [3315.4000000000005, 536.6725822204502, -0.0, -0.0];;], flex_in = [[-0.0, -0.0, 0.0]; [-0.0, -0.0, 0.0]; … ; [0.0, -0.0, 50.0]; [0.0, 0.0, 50.0];;], reg_in = [[-0.0]; [-0.0]; [2400.0];;], pol_out = [[2325.96, 205.79356638692835, -0.0, -0.0]; [0.0, 2648.2672292558204, -0.0, -0.0]; … ; [3590.77, 318.3522856110706, -0.0, -0.0]; [3315.4000000000005, 608.0197497360081, -0.0, -0.0];;], flex_out = [[-0.0, 0.0, 0.0]; [-0.0, 0.0, 0.0]; … ; [0.0, -142.26332462624353, 50.0]; [0.0, -142.26332462624353, 50.0];;], reg_out = [[-0.0]; [-0.0]; [2400.0];;]), bids = (B_pol_in = [2.385516244767792; 35.14551624476779; … ; 3.814636236794897; 7.834636236794893;;], B_pol_up_in = [0.0; 0.0; … ; 0.0; 0.0;;], B_pol_down_in = [2.385516244767792; 35.14551624476779; … ; 3.814636236794897; 7.834636236794893;;], B_flex_up_in = [0.0; 0.0; … ; 34.734193741279654; 11.66672

In [8]:
# We only look at the first fold
in_sample_Pw, out_sample_Pw, in_sample_prob, out_sample_prob, onS = sample_split(Pw[hour], nS_all, prob, fold_size, fold)

# We extract the intitial values from the benchmark, this will be our basis 
@unpack k0_up, k0_down, ppB0_up, ppB0_down, z0_up, z0_down, Imb_abs_0, s0, s_up0, s_down0 = extract_initial_values_from_benchmark(res_BM.history.history_insample[fold], fold_size)

base_kwargs = Dict{Symbol,Any}(
    :k0_up => k0_up,
    :k0_down => k0_down,
    :ppB0_up => ppB0_up,
    :ppB0_down => ppB0_down,
    :z0_up => z0_up,
    :z0_down => z0_down,
    :s_up0 => s_up0,
    :s_down0 => s_down0,
)

base_kwargs[:q_up] = q_up
base_kwargs[:q_down] = q_down
base_kwargs[:rf_up] = rf_up
base_kwargs[:rf_down] = rf_down
base_kwargs[:NI_up] = NI_up_og
base_kwargs[:NI_down] = NI_down_og

160

testing Bin intial values

In [9]:
k0_up_range  = collect(0.0:0.5:1.0)
k0_down_range  = collect(0.0:0.5:1.0)
z0_up_range = collect(0.0:NI_up_og/2:NI_up_og)
z0_down_range  = collect(0.0:NI_down_og/2:NI_down_og)
ppB0_up_range = collect(0.0:50.0:100.0)
ppB0_down_range  = collect(0.0:50.0:100.0)
s0_up_range = collect(0.0:1.0:1.0)
s0_down_range  = collect(0.0:1.0:1.0)

2-element Vector{Float64}:
 0.0
 1.0

Testing initial values on the Bin model

In [10]:
function run_model_experiments_OAT(
    Pw, nS, prob;
    model::Symbol,
    sweep_vars::Dict{Symbol,<:AbstractVector},
    base_kwargs = Dict())

    # ------------------------------------------------------------------
    # Two-sided One-At-A-Time (OAT) sensitivity analysis.
    #
    # For each sweep variable:
    #   - run a baseline at lower bounds
    #   - perturb the variable upward
    #   - run a baseline at upper bounds
    #   - perturb the variable downward
    #
    # Scalars are expanded to arrays using base_kwargs shapes.
    # ------------------------------------------------------------------

    results = []
    var_names = collect(keys(sweep_vars))

    # Helper: expand scalar to array if needed
    function expand(vname, vval)
        if haskey(base_kwargs, vname)
            return fill(vval, size(base_kwargs[vname])...)
        else
            return vval
        end
    end

    function run_and_store!(results, Pw, nS, prob, model;
                            kwargs,
                            sweep_vals,
                            baseline_side,
                            experiment,
                            active_var)

        println(kwargs)
        DA_bids, obj = run_model(Pw, nS, prob, model; kwargs...)

        result = Dict(
            :ppDA          => DA_bids[1],
            :pfDA          => DA_bids[2],
            :pgDA          => DA_bids[3],
            :objective     => obj,
            :baseline_side => baseline_side,   # :LB or :UB
            :experiment    => experiment,      # :baseline or :OAT
            :active_var    => active_var        # Symbol or :none
        )

        for (vname, vval) in sweep_vals
            result[vname] = vval
        end

        push!(results, result)
    end
    
    # --------------------------------------------------------------
    # LOWER-BOUND BASELINE
    # --------------------------------------------------------------
    baseline_lb_kwargs = deepcopy(base_kwargs)
    baseline_lb_vals   = Dict{Symbol,Any}()

    for (vname, vrange) in sweep_vars
        lb = first(vrange)
        baseline_lb_vals[vname] = lb
        baseline_lb_kwargs[vname] = expand(vname, lb)
        println("vname:", vname)
        println("vrange:", vrange)
    end

    println("→ Running baseline at LOWER bounds")
    println("baseline vals", baseline_lb_vals)
    run_and_store!(
        results, Pw, nS, prob, model;
        kwargs        = baseline_lb_kwargs,
        sweep_vals   = baseline_lb_vals,
        baseline_side = :LB,
        experiment    = :baseline,
        active_var    = :none)

    # OAT upward from lower bound
    for (vname, vrange) in sweep_vars
        lb = first(vrange)

        for vval in vrange
            vval == lb && continue

            println("→ OAT from LB: $vname = $vval")

            kwargs = deepcopy(baseline_lb_kwargs)
            kwargs[vname] = expand(vname, vval)

            sweep_vals = deepcopy(baseline_lb_vals)
            sweep_vals[vname] = vval

            run_and_store!(
                results, Pw, nS, prob, model;
                kwargs        = kwargs,
                sweep_vals   = sweep_vals,
                baseline_side = :LB,
                experiment    = :OAT,
                active_var    = vname
            )
        end
    end

    println("Running upper bounds now")

    # --------------------------------------------------------------
    # UPPER-BOUND BASELINE
    # --------------------------------------------------------------
    baseline_ub_kwargs = deepcopy(base_kwargs)
    baseline_ub_vals   = Dict{Symbol,Any}()

    for (vname, vrange) in sweep_vars
        ub = last(vrange)
        baseline_ub_vals[vname] = ub
        baseline_ub_kwargs[vname] = expand(vname, ub)
    end

    println("→ Running baseline at UPPER bounds")
    run_and_store!(
        results, Pw, nS, prob, model;
        kwargs        = baseline_ub_kwargs,
        sweep_vals   = baseline_ub_vals,
        baseline_side = :UB,
        experiment    = :baseline,
        active_var    = :none)

    # OAT downward from upper bound
    for (vname, vrange) in sweep_vars
        ub = last(vrange)

        for vval in vrange
            vval == ub && continue

            println("→ OAT from UB: $vname = $vval")

            kwargs = deepcopy(baseline_ub_kwargs)
            kwargs[vname] = expand(vname, vval)

            sweep_vals = deepcopy(baseline_ub_vals)
            sweep_vals[vname] = vval

            run_and_store!(
            results, Pw, nS, prob, model;
            kwargs        = kwargs,
            sweep_vals   = sweep_vals,
            baseline_side = :UB,
            experiment    = :OAT,
            active_var    = vname
            )
        end
    end

    return results
end


run_model_experiments_OAT (generic function with 1 method)

In [15]:
# Variables set to lower bounds. 
results = run_model_experiments_OAT(in_sample_Pw, fold_size, in_sample_prob; model =:bin,
    sweep_vars = Dict(:k0_up => k0_up_range, :k0_down => k0_down_range),
    base_kwargs=base_kwargs)

objectives = getindex.(results, :objective)
std(objectives)

vname:k0_up
vrange:[0.0, 0.5, 1.0]
vname:k0_down
vrange:[0.0, 0.5, 1.0]
→ Running baseline at LOWER bounds
baseline valsDict{Symbol, Any}(:k0_up => 0.0, :k0_down => 0.0)
Dict{Symbol, Any}(:s_up0 => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :k0_down => [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0], :z0_down => [123.34000000000002; 89.2; 131.01999999999998; 149.74000000000004; 125.85999999999999; 148.90000000000003; 115.66000000000003; 159.99999999999997; 110.56000000000002; 113.07999999999996;;], :rf_down => [0.0, 0.0, 0.0, 0.0, 40.0, 40.0,

9.701276818911234e-12

In [20]:
ppDAs = getindex.(results, :ppDA)
ppDAs_std_devs = round.(mapslices(std, ppDAs; dims=1), digits=2)  # std along each row
ppDAs_means = round.(mapslices(mean, ppDAs; dims=1), digits=2)  # mean along each row

pfDAs = getindex.(results, :pfDA)
pfDAs_std_devs = round.(mapslices(std, pfDAs; dims=1), digits=2)  # std along each row
pfDAs_means = round.(mapslices(mean, pfDAs; dims=1), digits=2)  # mean along each row

pgDAs = getindex.(results, :pgDA)
pgDAs_std_devs = round.(mapslices(std, pgDAs; dims=1), digits=2)  # std along each row
pgDAs_means = round.(mapslices(mean, pgDAs; dims=1), digits=2)  # mean along each row

println(ppDAs_std_devs)
println(pfDAs_std_devs)
println(pgDAs_std_devs)
println("Means:")
println(ppDAs_means)
println(pfDAs_means)
println(pgDAs_means)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
Means:
[32.76, 32.76, 32.76, 32.76, 32.76, 32.76, 52.42, 52.42, 52.42, 52.42, 52.42, 52.42]
[0.0, 0.0, 0.0, 0.0, 88.92, 450.0, 450.0, 450.0]
[0.0, 0.0, 50.0]


In [22]:

using DataFrames, CSV, JLD2

outdir = joinpath(@__DIR__, "export_HPC")
mkpath(outdir)

data = Dict(
    "results" => results,
    "meta" => Dict(
        "hour" => hour,
        "fold" => fold,
        "fold_size" => fold_size,
        "model" => :bin,
    )
)

@save joinpath(outdir, "OAT_sensitivity_results.jld2") data
